# Prediction with ForceRegression model

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from src.vct_scaling import scale_force_to_model_scale, scale_moment_to_model_scale


In [ ]:
id = "22773"
df = catalog.load(f"{id}.data")
df_smooth = catalog.load(f"{id}.data_ek_smooth")
model = catalog.load("vmm_martins_simple.force_regression.model")
regression = catalog.load("vmm_martins_simple.force_regression.regression")
df_VCT = catalog.load("force_regression.data_scaled")
ship_data = catalog.load("ship_data")

model_motion_regression = catalog.load("vmm_martins_simple.motion_regression.joined.model")

#keys_forces = ['fx','fy','fz']
#scale_factor=ship_data['scale_factor']
#forces=df_VCT[keys_forces]
#df_VCT[keys_forces] = scale_force_to_model_scale(forces=forces, scale_factor=scale_factor, **df_VCT)
#keys_moments = ['mx','my','mz']
#df_VCT[keys_moments] = scale_force_to_model_scale(df_VCT[keys_moments], scale_factor=scale_factor, **df_VCT)


In [ ]:
df_VCT[['u','v','V','r','fx','fy','mz']].abs().max()

In [ ]:
model.parameters

In [ ]:
#model.parameters['Xu'] = -0.000737*1.7
result = model.simulate(df_=df_smooth)

In [ ]:
result.result.plot(y='U')

In [ ]:
dataframes = {
'model test' : df_smooth,
'simulation (model test)' : result.result,    
}

fig,ax=plt.subplots()
track_plots(dataframes=dataframes, lpp=model.ship_parameters['L'], beam=model.ship_parameters['B'], ax=ax, plot_boats=False);

plot(dataframes, keys=result.result.columns);

In [ ]:
data = df_VCT[['fx','fy','mz','u','v','V','r','beta','delta','thrust','test type']].copy()
df_model = data.copy()
df_model[['fx','fy','mz']] = model.forces(inputs=df_model)
df_model['item'] = 'regression'

In [ ]:
df = data.copy()
df['item'] = 'VCT'
df = df.append(df_model)

captive_plot(df, styles=['.','-'])

In [ ]:
from wPCC_pipeline.pipelines.prediction.nodes import track_plot

In [ ]:
vmm = "vmm_martin"
regression = "motion_regression"
id = 22637
results = catalog.load(f"vmm_martin.motion_regression.22637.data_resimulate")
data = catalog.load("22637.data_ek_smooth")
ship_data = catalog.load("ship_data")

track_plot(data=data, results=results, ship_data=ship_data);

In [ ]:
#id = 22612
id = 22613
data_raw = catalog.load(f"{id}.raw_data")
data_lowpass = catalog.load(f"{id}.data_lowpass")
data_ek_smooth = catalog.load(f"{id}.data_ek_smooth")
data_ek_filter = catalog.load(f"{id}.data_ek_filter")
data = catalog.load(f"{id}.data")

fig,ax=plt.subplots()

data_lowpass.plot(y='u', label='lowpass', ax=ax)
data.plot(y='u', label='data', ax=ax)
data_ek_smooth.plot(y='u', label='smooth', ax=ax)
data_ek_filter.plot(y='u', label='EKF', ax=ax)

data_raw.index-=data_raw.index[0]
data_raw['dx0'] = np.gradient(data_raw['x0'],data_raw.index)
data_raw.rolling(window=20).mean().plot(y='dx0', label='raw', ax=ax, zorder=-10)

ax.set_ylim(0.9,1.05)

In [ ]:
from wPCC_pipeline.pipelines.preprocess.nodes import filter, assemble_data, add_thrust
from wPCC_pipeline.pipelines.extended_kalman.nodes import create_extended_kalman
from wPCC_pipeline.pipelines.brix.nodes import initial_parameters
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter,extended_kalman_smoother

In [ ]:
fig,ax=plt.subplots()
thrust_channels = catalog.load("params:thrust_channels")
data_with_thrust = add_thrust(df=data_raw, thrust_channels=thrust_channels)
df_lowpass = filter(df=data_with_thrust, cutoff=10, order=1)

df_lowpass.plot(y='u', label='lowpass', ax=ax)

In [ ]:
df_data = assemble_data(df_lowpass=df_lowpass, raw_data=data_raw)

In [ ]:
ship_data = catalog.load("ship_data")
vmm = catalog.load("vmm_martin")
                   
parameters = initial_parameters(ship_data=ship_data)

ek = create_extended_kalman(parameters=parameters, ship_data=ship_data, vmm = vmm) 

In [ ]:
variance_u = 0.1
variance_v = 0.1
variance_r = np.deg2rad(0.1)
Qd = np.diag([variance_u, variance_v, variance_r])  # process variances: u,v,r

error_max_pos = 0.05
sigma_pos = error_max_pos / 3
variance_pos = sigma_pos ** 2

error_max_psi = np.deg2rad(0.5)
sigma_psi = error_max_psi / 3
variance_psi = sigma_psi ** 2

Rd = np.diag([variance_pos, variance_pos, variance_psi])
P_prd = np.diag(
    [
        variance_pos,
        variance_pos,
        variance_psi,
        variance_u,
        variance_v,
        variance_r,
    ]
)

covariance_matrixes = {
        "P_prd": P_prd.tolist(),
        "Qd": Qd.tolist(),
        "Rd": Rd.tolist(),
    }


ek_filtered, data_ek_filter = extended_kalman_filter(ek=ek, data=df_data, covariance_matrixes=covariance_matrixes)

In [ ]:
ek_smooth, data_ek_smooth = extended_kalman_smoother(ek=ek)

In [ ]:
fig,ax=plt.subplots()

df_lowpass.plot(y='u', label='lowpass', ax=ax)
df_data.plot(y='u', label='data', ax=ax)
data_ek_filter.plot(y='u', label='EKF', ax=ax)
data_ek_smooth.plot(y='u', label='smooth', ax=ax)

#ax.set_ylim(0.9,1.05)